*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(train_file_path, sep="\t", names=['Class', 'Message'])
test_dataset = pd.read_csv(test_file_path, sep="\t", names=['Class', 'Message'])
# train_dataset.head()

In [ ]:
train_dataset['Class'] = pd.factorize(train_dataset['Class'])[0]
test_dataset['Class'] = pd.factorize(test_dataset['Class'])[0]
# train_dataset.head()
# ham ==0, spam ==1

In [ ]:
train_labels = train_dataset.pop('Class')
test_labels = test_dataset.pop('Class')
# test_labels.head()

In [ ]:
max_len = 50
trunc_type = 'post'
padd_type = 'post'
oov_token = '<OOV>'
vocab_size = 7000    # set this up to have access to all the tokenized words
epochs = 30

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_token)
tokenizer.fit_on_texts(train_dataset['Message'])

In [ ]:
word_index = tokenizer.word_index
# word_index # visualize all tokens 

In [ ]:
train_sequence = tokenizer.texts_to_sequences(train_dataset['Message'])
train_padd = pad_sequences(train_sequence, maxlen = max_len,
                           padding = padd_type, truncating = trunc_type )

test_sequence = tokenizer.texts_to_sequences(test_dataset['Message'])
test_padd = pad_sequences(test_sequence, maxlen = max_len,
                               padding = padd_type, truncating = trunc_type)
# index = 1
# print(train_sequence[index])               ### visualize passed data
# print(train_padd[index])
# train_dataset['Message'][index]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 16, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

In [ ]:
history = model.fit(train_padd, train_labels, epochs= epochs,
                   validation_data=(test_padd, test_labels), verbose=0)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    new_token = tokenizer.texts_to_sequences([pred_text])
    padd = pad_sequences(new_token, maxlen = max_len,
                       padding = padd_type, truncating = trunc_type)
    prob = model.predict(padd)
    prediction = [prob[0][0], 'ham' if prob < .5 else 'spam']
    return prediction
# the test_prediction will loop through a test_messages and return a string.
## Tokenizer can only receive a list (itirable argument).
### don't need to loop through anything, test_prediction does it for us.

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
